In [1]:
# This code reads the text content and identifies the feature names and related values.
from os import listdir
from os.path import isfile, join
import pandas as pd
from pandas import ExcelWriter
import ast





def extract_information(file):
    with open(file,mode="r") as ins:
        data = []
        mls_number = ""

        state = None
        for line in ins:

            if line == "Listing Summary:\n":
                state = "LS_Parse_Next_Line"
                continue
            elif line == "Tax History:\n":
                state = "PH_Skip_Next_Then_Parse_Next_Line"
                continue

            if state == "LS_Parse_Next_Line":
                entries = eval(line)  # Read the line as JSON array.
                for entry in entries:
                    if entry[0] == "MLS#":
                        mls_number = entry[1]

                state = None

            elif state == "PH_Parse_Next_Line":
                data = eval(line)
                state = None

            elif state == "PH_Skip_Next_Then_Parse_Next_Line":
                state = "PH_Parse_Next_Line"

        for row in data:
            row.insert(0, mls_number)

        return data


mypath = "C:\\Users\\39118\\Desktop\\Python Program\\LIINFO"  # Change this to your own directory
list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]  # Store all file names in a vector

headers = ['a', 'b', 'c', 'd', 'e', 'f']

rows = []
count = 0
for file in list_files:
    data = extract_information("C:\\Users\\39118\\Desktop\\Python Program\\LIINFO\\" + file)
    rows = rows + data

df = pd.DataFrame.from_records(rows, columns=headers)
# df = pd.DataFrame({'a': headers, 'b': rows})
# df = df.transpose()  # switches columns into rows
with pd.ExcelWriter('C:\\Users\\39118\Desktop\\PYTHON CODE\\Tax Data222.xlsx') as writer:
    df.to_excel(writer, 'sheet1',startrow=0, startcol=0, header=True, index=False)